In [2]:
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 2.3 MB/s eta 0:00:0000:0100:01


In [23]:
import os
from glob import glob
import numpy as np
from scipy.io import loadmat
import rasterio as rio
from rasterio import mask
import geopandas as gpd
from rasterio.plot import show

from sklearn import cluster
from osgeo import gdal, gdal_array
import matplotlib.pyplot as plt

# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()

In [ ]:
############# to do --> read data function ##################################
root_directory = '/home/jovyan/output'

image_list = []

for path, subdirs, files in os.walk(root_directory):
    print(path)  
    dirname = path.split(os.path.sep)[-1]
    #if dirname == 'R10m':   #Find all 'images' directories
    #if dirname == 'masks':   #Find all 'images' directories
    #if dirname == 'data':   #Find all 'images' directories
    if dirname == 'output':   #Find all 'images' directories
        images = os.listdir(path)  #List of all image names in this subdirectory
        for i, image_name in enumerate(images):  
            if image_name.endswith("0212.tif"): 
               #print(image_name)
               image_list.append(image_name)
               #image_name = rio.open(root_directory+'/'+image_name)
image_list
#############################################################################

In [ ]:
for image in image_list:
    print (image)
    S_sentinel_bands = glob(root_directory+'/'+image)
#S_sentinel_bands = glob("/home/jovyan/output/sololo_20240212.tif")
    S_sentinel_bands.sort()
    print(S_sentinel_bands)
#S_sentinel_bands = clipped_bands
    l = []
    for i in S_sentinel_bands:
      with rio.open(i, 'r') as f:
          print (i)
          l.append(f.read(1))
          arr_st = np.stack(l)
          for i in S_sentinel_bands:
            img_ds = gdal.Open(i, gdal.GA_ReadOnly)
            
            img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
                          gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))
            
            for b in range(img.shape[2]):
                img[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()

            new_shape = (img.shape[0] * img.shape[1], img.shape[2])
            X = img[:, :, :4].reshape(new_shape)
            k_means = cluster.KMeans(n_clusters=10, n_init=10)
            k_means.fit(X)
            
            X_cluster = k_means.labels_
            X_cluster = X_cluster.reshape(img[:, :, 0].shape)

            plt.figure(figsize=(20,20))
            plt.imshow(X_cluster, cmap="tab10")
            plt.title("classified "+image, fontsize=16, fontweight="bold")
            plt.show()

            output_file = root_directory+'/'+"classified_"+image
            with rio.open(output_file, 'w', driver='GTiff', height=height, width=width, count=count, dtype=dtype, crs=f.crs, transform=transform) as dst:
                dst.write(X_cluster, 1)

In [41]:
f.crs

CRS.from_epsg(4326)

In [47]:
############# to do --> read data function ##################################
root_directory = '/home/jovyan/output'

image_list = []

for path, subdirs, files in os.walk(root_directory):
    print(path)  
    dirname = path.split(os.path.sep)[-1]
 
    if dirname == 'output':   #Find all 'images' directories
        images = os.listdir(path)  #List of all image names in this subdirectory
        for i, image_name in enumerate(images):  
            if image_name.startswith("classified"): 
               #print(image_name)
               image_list.append(image_name)
               #image_name = rio.open(root_directory+'/'+image_name)
image_list
#############################################################################

/home/jovyan/output
/home/jovyan/output/.ipynb_checkpoints


['classified_sololo_20240212.tif']

In [187]:
from osgeo import gdal, ogr

def vectorize_gdal_ogr(filepath, output_path):
    """
    Vectorizes a single polygon GeoTIFF using GDAL and OGR and saves as a GeoPackage.

    Args:
        filepath (str): Path to the input GeoTIFF file.
        output_path (str): Path to save the output GeoPackage file.
    """

    # Open the GeoTIFF image as a raster dataset
    dataset = gdal.Open(filepath)
    if dataset is None:
        raise OSError(f"Failed to open GeoTIFF: {filepath}")
    else:
        print("Raster loaded")

    # Create a new vector driver (GPKG format)
    driver = ogr.GetDriverByName("GPKG")

    # Create a new GeoPackage
    output_datasource = driver.CreateDataSource(output_path)
    if output_datasource is None:
        raise OSError(f"Failed to create GeoPackage: {output_path}")

    # Create a new layer in the GeoPackage
    spatial_ref = dataset.GetSpatialRef()
    layer = output_datasource.CreateLayer("vectorized", geom_type=ogr.wkbPolygon)
    #diss_layer = output_datasource.CreateLayer("dissolved", geom_type=ogr.wkbPolygon)

    # Create a new field for storing any desired attribute (optional)
    field_name = "Value"  # Replace with the correct field name
    field_defn = ogr.FieldDefn(field_name, ogr.OFTInteger)
    layer.CreateField(field_defn)

    # Perform polygonization
    src_band = dataset.GetRasterBand(1)
    gdal.Polygonize(src_band, maskBand=None, outLayer=layer, iPixValField=0, options=[], callback=None)

    # Check if the vectorized layer contains features
    feature_count = layer.GetFeatureCount()
    print("Number of features in vectorized layer:", feature_count)

    # Get the actual field names from the layer
    layer_defn = layer.GetLayerDefn()
    field_names = [layer_defn.GetFieldDefn(i).GetName() for i in range(layer_defn.GetFieldCount())]

    # Check distinct values in the specified field
    distinct_values = set()
    if field_name in field_names:
        for feature in layer:
            value = feature.GetField(field_name)
            distinct_values.add(value)
        print("Distinct values in field 'Value':", distinct_values)

        # Dissolve the polygons based on the specified field
        for value in distinct_values:
            sql = f"SELECT ST_union(geometry) AS geometry FROM {layer.GetName()} WHERE {field_name} = {value}"
            print("SQL Query:", sql)
            dissolved_features = output_datasource.ExecuteSQL(sql)
            if dissolved_features is not None:
                for feature in dissolved_features:
                    dissolved_layer.CreateFeature(feature)
    else:
        print(f"Field '{field_name}' not found in the vectorized layer.")

    # Cleanup
    output_datasource = None
    dataset = None

# Example usage (modify filepaths)
filepath = '/home/jovyan/output/classified_sololo_20240212.tif'
output_path = '/home/jovyan/output/output.gpkg'

vectorize_gdal_ogr(filepath, output_path)

Raster loaded
Number of features in vectorized layer: 378670
Distinct values in field 'Value': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
SQL Query: SELECT ST_union(geometry) AS geometry FROM vectorized WHERE Value = 0
SQL Query: SELECT ST_union(geometry) AS geometry FROM vectorized WHERE Value = 1
SQL Query: SELECT ST_union(geometry) AS geometry FROM vectorized WHERE Value = 2
SQL Query: SELECT ST_union(geometry) AS geometry FROM vectorized WHERE Value = 3
SQL Query: SELECT ST_union(geometry) AS geometry FROM vectorized WHERE Value = 4
SQL Query: SELECT ST_union(geometry) AS geometry FROM vectorized WHERE Value = 5
SQL Query: SELECT ST_union(geometry) AS geometry FROM vectorized WHERE Value = 6
SQL Query: SELECT ST_union(geometry) AS geometry FROM vectorized WHERE Value = 7
SQL Query: SELECT ST_union(geometry) AS geometry FROM vectorized WHERE Value = 8
SQL Query: SELECT ST_union(geometry) AS geometry FROM vectorized WHERE Value = 9


In [126]:
! gdalinfo --version

GDAL 3.4.1, released 2021/12/27


In [ ]:
################### maybe useful #####################################################

In [119]:
import rasterio
from rasterio.features import shapes
from shapely.geometry import mapping  # for converting geometry to dict
import geopandas as gpd

def vectorize_and_dissolve_by_class(geotiff_path, gpkg_path):
  """
  Vectorizes a GeoTIFF image, assigns a "class" attribute based on pixel values,
  dissolves polygons based on the "class", and saves the result as a GeoPackage (.gpkg) file.

  Args:
      geotiff_path (str): Path to the GeoTIFF image file.
      gpkg_path (str): Path to the output GeoPackage file.
  """

  # Open the GeoTIFF dataset
  with rasterio.open(geotiff_path) as src:
      # Get image data, metadata, and number of bands
      image = src.read()
      profile = src.profile
      n_bands = image.shape[0]

  # Define a dictionary to map pixel values to classes (modify as needed)
  class_mapping = {
      1: "water",
      2: "vegetation",
      3: "urban",
      4: "4",
      5: "5"# Example mapping, replace with your actual class labels
  }

  # Function to assign class based on pixel value
  def get_class(pixel_values):
    # Assuming a single-band image, use the first element of the tuple
    if n_bands == 1:
      return class_mapping.get(pixel_values[0], "unknown")  # Handle unknown values
    else:
      # Adapt logic for multi-band images based on your classification scheme
      raise NotImplementedError("Multi-band class assignment not implemented. Modify the logic based on your classification scheme.")

  # Vectorize the image with custom attribute assignment
  vector_shapes = []
  for val, shape in shapes(image, mask=None, transform=profile['transform'], callback=get_class):
      # Add a "properties" dictionary with the assigned class
      shape['properties'] = {'class': val}
      vector_shapes.append(shape)

  # Convert shapes to GeoDataFrame with geometry and class
  vector_gdf = gpd.GeoDataFrame({ 'geometry': [mapping(shape['geometry']) for shape in vector_shapes] })
  vector_gdf['class'] = [shape['properties']['class'] for shape in vector_shapes]

  # Dissolve the GeoDataFrame based on the "class" property
  dissolved_gdf = vector_gdf.dissolve(by='class', as_index=False)

  # Save the dissolved GeoDataFrame to a GeoPackage
  dissolved_gdf.to_file(driver='GPKG', filename=gpkg_path, layer='dissolved_polygons')
  print(f"GeoTIFF vectorized, dissolved, and saved as: {gpkg_path}")

# Example usage
geotiff_path = "/home/jovyan/data/classified_eichenberg.tiff"
gpkg_path = "/home/jovyan/data/classified_eichenberg.gpkg"

vectorize_and_dissolve_by_class(geotiff_path, gpkg_path)


print("GeoTIFF vectorized and saved as", gpkg_path)


TypeError: shapes() got an unexpected keyword argument 'callback'

({'type': 'Polygon', 'coordinates': [[(563544.6907501516, 5692731.900746037), (563544.6907501516, 5692731.850747752), (563544.7407486883, 5692731.850747752), (563544.7407486883, 5692731.900746037), (563544.6907501516, 5692731.900746037)]]}, 4.0)


In [114]:
geotiff_path = "/home/jovyan/data/classified_eichenberg.tiff"
with rasterio.open(geotiff_path) as src:
  # Get image data and metadata
  image = src.read(1)
  profile = src.profile

  # Convert raster data to vector shapes
vector_shapes = shapes(image, mask=None, transform=profile['transform'])

In [118]:
profile

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 16991, 'height': 12757, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(0.04999853667642206, 0.0, 563150.9522738248,
       0.0, -0.04999828499509282, 5692731.900746037), 'blockysize': 1, 'tiled': False, 'interleave': 'band'}

In [115]:
for first_shape in vector_shapes:
    print (first_shape)
    break

({'type': 'Polygon', 'coordinates': [[(563544.1407662482, 5692731.900746037), (563544.1407662482, 5692731.850747752), (563544.1907647849, 5692731.850747752), (563544.1907647849, 5692731.900746037), (563544.1407662482, 5692731.900746037)]]}, 2.0)


In [117]:
 vector_gdf = gpd.GeoDataFrame({ 'geometry': [mapping(shape[0]['coordinates']) for shape in vector_shapes] })

AttributeError: 'list' object has no attribute '__geo_interface__'